In [ ]:
with open('company.csv','r') as f:
    c = f.readlines()
companies =  ordered_set.OrderedSet()

for item in c[1:]:
    companies.add(item.strip('\n", '))
    
syn=[(),
    ('aktiengesellschaft','ag'),
    ('Limited liability company','llc'),('Public limited company','plc'),('incorporation','inc'),
    ('Sociedad por Acciones','sa'),('Public Limited','ltd'),('int','international'), ('london','bishopsgate (110)'),
    ('paris','fort dup 11046'),('london','great winche'),('london','golden lane'),('new world development','nwd'),
    ('england ','bishopsgate (110)'),
    ('london','bishopsgate (110)'),
    ('acting through its london branch','great winche'),
    ('acting through its london branch','golden lane'),
    ('incorporated with limited liability in england and wales','bishopsgate (110)'),
    ('rubbermaid inc','brands inc'),
    ('dexia funding netherlands','belfius financing company sa'),
    ('province of british columbia','government of the province of british columbia'),
    ('london branch','one cabot square')]

fcomp = fuzzCompany2(companies,syn)

In [1]:
from code.Matching import fuzzCompany2
import os
from code import Text_reader as tr
from code.Labels import label
from code.Text_reader import htmlDoc
import copy 
import re
from code.dataExtract import data
import code.dataExtract as de
import numpy as np

d = data(folder='../Newdata/train/')

fcomp = fuzzCompany2.from_file('fc_data2.pkl')

syn=[(),
    ('aktiengesellschaft','ag'),
    ('Limited liability company','llc'),('Public limited company','plc'),('incorporation','inc'),
    ('Sociedad por Acciones','sa'),('Public Limited','ltd'),('int','international'), ('london','bishopsgate (110)'),
    ('paris','fort dup 11046'),('london','great winche'),('london','golden lane'),('new world development','nwd'),
    ('england ','bishopsgate (110)'),
    ('london','bishopsgate (110)'),
    ('acting through its london branch','great winche'),
    ('acting through its london branch','golden lane'),
    ('incorporated with limited liability in england and wales','bishopsgate (110)'),
    ('rubbermaid inc','brands inc'),
    ('dexia funding netherlands','belfius financing company sa'),
    ('province of british columbia','government of the province of british columbia'),
    ('london branch','one cabot square')]

The PythonMagick package is missing, get_text_adv function can not be used 


In [2]:
'ONE CABOT SQUARE'.lower()

'one cabot square'

In [3]:
#! NEW ONE !!!
def get_issuer(txt):
    txtc = re.sub("[^a-zA-Z0-9\s]", 
                      "",                   
                      txt)
    txtcn=txtc.replace('\n',' ').replace('\r','').lower()
    v=['+']
    tmp=[]
    
    for item in syn:
        if len(item)==0:
            txtc = txtcn.split()
        else:
            txtc = txtcn.replace(item[0],item[1]).split()
        #print txtc
        #print '================================'
        for x in xrange(0,len(txtc)):
            if txtc[x] in fcomp.Lookup:
                tmp.append(fcomp.Lookup[txtc[x]])
            else:
                if len(tmp) > 0:
                    v.append(tmp)
                    tmp=[]
                if isinstance(v[-1],int):
                    v[-1] -= 1
                else:
                    v.append(-1)

            if txtc[x] in ['issue','issued']:  
                if not isinstance(v[-1],int):
                    v.append(0)
                v.append('-')

            if len(txtc)> x+1 and txtc[x] == 'linked' and txtc[x+1] == 'to':
                v.append('*')

            if len(txtc)> x+1 and txtc[x] == 'share' and txtc[x+1] == 'issuer':
                v.append('*')
            
            if len(txtc)> x+1 and txtc[x] == 'equity' and txtc[x+1] == 'issuer':
                v.append('*')
                
            if len(txtc)> x+1 and txtc[x] == 'guarantor':
                v.append('*')
              
            if len(txtc)> x+1 and txtc[x] == 'dealer':
                v.append('*')
             
            if len(txtc)> x+4 and txtc[x] == 'of' and txtc[x+1] == 'the' and txtc[x+2] == 'issue' and txtc[x+3] == 'of' and txtc[x+4] == 'securities':
                v.append('*')   
            
            if len(txtc)> x+1 and txtc[x] == 'lead' and txtc[x+1] == 'manager':
                v.append('*') 
                
            if txtc[x] in ['issuer','issued']:
                v.append('+')
                v.append(0)

    group=set()
    x=0
    while x<len(v):
        if v[x]=='*' and x+2 < len(v):
            x+=2
            continue
        if v[x]=='-' and x > 2:
            group.add((abs(v[x-1]),tuple(v[x-2])))
        if v[x]=='+' and x+2 < len(v):
            if isinstance(v[x+1],int):
                group.add((abs(v[x+1]),tuple(v[x+2]))) 
            else:
                if isinstance(v[x+2],int):
                    group.add((abs(v[x+2]),tuple(v[x+3])))
                else:
                    group.add((0,tuple(v[x+2])))
        x+=1

    r=[]

    x=0
    for item in group:
        subset = fcomp.get_subsets(item[1])
        ps = fcomp.get_Ps_vect(subset)
        idx = np.argmax(ps)
        #print x,fcomp.values[idx], ps[idx]/float(item[0]+1)*len(fcomp.values[idx].split()),item[0]
        x+=1
        r.append((ps[idx]/float(item[0]+1)*len(fcomp.values[idx].split()), fcomp.values[idx]))

    r = sorted(r,key=lambda x:x[0],reverse=True)
    if len(r)>0:
        return r[0][1]
    else:
        return None


In [4]:
folder='../Newdata/train/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../Newdata/train/txt')

issuerDic={}

x=0
for isin in d.docid: # for each isin in the data object created before
#for isin in ['CA11069ZCW55']:
    if x > 300:
        break
    x+=1
    
    f=d.docid[isin] # get the file_ids
    issuer=None
    for item in f: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        issuer=get_issuer(txt)
        if issuer is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            issuer=get_issuer(txt)
            
        if issuer is not None:
            break
    
    print isin, issuer
    if issuer is not None:
        issuerDic[isin]=issuer
    else:
        issuerDic[isin]=''

XS1378463068 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1267246327 GOLDMAN SACHS INTERNATIONAL
XS1314875508 SG ISSUER
US50066RAB24 KOREA NATIONAL OIL CORPORATION
XS1398577475 HSBC BANK PLC
XS1346593194 UBS AG, GOLDEN LANE
XS1343808702 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1254344952 CREDIT SUISSE INTERNATIONAL
CH0295931650 EXANE FINANCE
XS1326554562 ROYAL BANK OF CANADA
XS1333969266 CREDIT SUISSE INTERNATIONAL
XS1402911298 ASIAN DEVELOPMENT BANK
XS1354249945 COMMERZBANK AG
CH0287822685 EFG INTERNATIONAL FINANCE (GUERNSEY) LIMITED
XS1313801786 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1305149954 EFG-HERMES MENA SECURITIES LIMITED
XS1274660338 DEUTSCHE BANK AG
XS1337149105 SG ISSUER
XS1358065362 HSBC BANK PLC
XS0536851537 BARCLAYS BANK PLC
XS1267285572 GOLDMAN SACHS INTERNATIONAL
XS1378444944 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
CH0279927146 EFG INTERNATIONAL FINANCE (GUERNSEY) LIMITED
XS1378425059 THE HONGKONG AND SHANGHAI BANKING CORPORATION LI

In [5]:
issuerAns = {}
x=0
for item in d.issuer:
    if item in issuerDic:    
        issuerAns[item] = d.issuer[item][0]
    
de.test(issuerDic, issuerAns)


total number of set compared: 301.0
total number of missing set: 0.0
error %: 9.3023255814


In [6]:
for item in issuerDic:
    if issuerDic[item] != issuerAns[item] and issuerDic[item]!='':
        print item, d.docid[item],' , ',issuerDic[item],' , ',issuerAns[item]

US60786LAB36 OrderedSet(['0900045c8404d701'])  ,  KOMMUNALBANKEN AS  ,  MODUSLINK GLOBAL SOLUTIONS, INC.
XS1353957225 OrderedSet(['0900045c83cb2c98'])  ,  MORGAN STANLEY  ,  MORGAN STANLEY & CO INTERNATIONAL PLC
XS1360532219 OrderedSet(['0900045c83d182a3'])  ,  BANK HAPOALIM BM  ,  HAPOALIM INTERNATIONAL NV
DE000A1KRG61 OrderedSet(['0900045c83de22df'])  ,  SG ISSUER  ,  FMS WERTMANAGEMENT
XS1069359716 OrderedSet(['0900045c838cb455'])  ,  BARCLAYS BANK PLC  ,  BARCLAYS BANK PLC, BISHOPSGATE (110)
CAU60901AC66 OrderedSet(['0900045c83af6949'])  ,  MOLSON COORS BREWING COMPANY  ,  MOLSON COORS INTERNATIONAL LP
XS0536851537 OrderedSet(['0900045c808dacd7'])  ,  BARCLAYS BANK PLC  ,  BARCLAYS BANK PLC, BISHOPSGATE (110)
CH0034478864 OrderedSet(['0900045c80bab4ad'])  ,  UBS AG  ,  UBS AG, GOLDEN LANE
DK0030370028 OrderedSet(['0900045c83a804d4'])  ,  KOMMUNALBANKEN AS  ,  TOPDANMARK A/S
USP26054AA76 OrderedSet(['0900045c83908ae8'])  ,  THE BANK OF NEW YORK MELLON (IRELAND) LIMITED  ,  CIBANCO S

Testing set
-------------

In [8]:
from code.Matching import fuzzCompany2
import os
from code import Text_reader as tr
from code.Labels import label
from code.Text_reader import htmlDoc
import copy 
import re
from code.dataExtract import data
import code.dataExtract as de
import numpy as np

d = data(folder='../Newdata/int_test/')

fcomp = fuzzCompany2.from_file('fc_data2.pkl')

syn=[(),
    ('aktiengesellschaft','ag'),
    ('Limited liability company','llc'),('Public limited company','plc'),('incorporation','inc'),
    ('Sociedad por Acciones','sa'),('Public Limited','ltd'),('int','international'), ('london','bishopsgate (110)'),
    ('paris','fort dup 11046'),('london','great winche'),('london','golden lane'),('new world development','nwd'),
    ('england ','bishopsgate (110)'),
    ('london','bishopsgate (110)'),
    ('acting through its london branch','great winche'),
    ('acting through its london branch','golden lane'),
    ('incorporated with limited liability in england and wales','bishopsgate (110)'),
    ('rubbermaid inc','brands inc'),
    ('dexia funding netherlands','belfius financing company sa'),
    ('province of british columbia','government of the province of british columbia'),
    ('london branch','one cabot square')]

In [9]:
#! NEW ONE !!!
def get_issuer(txt):
    txtc = re.sub("[^a-zA-Z0-9\s]", 
                      "",                   
                      txt)
    txtcn=txtc.replace('\n',' ').replace('\r','').lower()
    v=['+']
    tmp=[]
    
    for item in syn:
        if len(item)==0:
            txtc = txtcn.split()
        else:
            txtc = txtcn.replace(item[0],item[1]).split()
        #print txtc
        #print '================================'
        for x in xrange(0,len(txtc)):
            if txtc[x] in fcomp.Lookup:
                tmp.append(fcomp.Lookup[txtc[x]])
            else:
                if len(tmp) > 0:
                    v.append(tmp)
                    tmp=[]
                if isinstance(v[-1],int):
                    v[-1] -= 1
                else:
                    v.append(-1)

            if txtc[x] in ['issue','issued']:  
                if not isinstance(v[-1],int):
                    v.append(0)
                v.append('-')

            if len(txtc)> x+1 and txtc[x] == 'linked' and txtc[x+1] == 'to':
                v.append('*')

            if len(txtc)> x+1 and txtc[x] == 'share' and txtc[x+1] == 'issuer':
                v.append('*')
            
            if len(txtc)> x+1 and txtc[x] == 'equity' and txtc[x+1] == 'issuer':
                v.append('*')
                
            if len(txtc)> x+1 and txtc[x] == 'guarantor':
                v.append('*')
              
            if len(txtc)> x+1 and txtc[x] == 'dealer':
                v.append('*')
             
            if len(txtc)> x+4 and txtc[x] == 'of' and txtc[x+1] == 'the' and txtc[x+2] == 'issue' and txtc[x+3] == 'of' and txtc[x+4] == 'securities':
                v.append('*')   
            
            if len(txtc)> x+1 and txtc[x] == 'lead' and txtc[x+1] == 'manager':
                v.append('*') 
                
            if txtc[x] in ['issuer','issued']:
                v.append('+')
                v.append(0)

    group=set()
    x=0
    while x<len(v):
        if v[x]=='*' and x+2 < len(v):
            x+=2
            continue
        if v[x]=='-' and x > 2:
            group.add((abs(v[x-1]),tuple(v[x-2])))
        if v[x]=='+' and x+2 < len(v):
            if isinstance(v[x+1],int):
                group.add((abs(v[x+1]),tuple(v[x+2]))) 
            else:
                if isinstance(v[x+2],int):
                    group.add((abs(v[x+2]),tuple(v[x+3])))
                else:
                    group.add((0,tuple(v[x+2])))
        x+=1

    r=[]

    x=0
    for item in group:
        subset = fcomp.get_subsets(item[1])
        ps = fcomp.get_Ps_vect(subset)
        idx = np.argmax(ps)
        #print x,fcomp.values[idx], ps[idx]/float(item[0]+1)*len(fcomp.values[idx].split()),item[0]
        x+=1
        r.append((ps[idx]/float(item[0]+1)*len(fcomp.values[idx].split()), fcomp.values[idx]))

    r = sorted(r,key=lambda x:x[0],reverse=True)
    if len(r)>0:
        return r[0][1]
    else:
        return None


In [10]:
folder='../Newdata/int_test/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../Newdata/int_test/txt')

issuerDic={}

for isin in d.docid: # for each isin in the data object created before
    
    f=d.docid[isin] # get the file_ids
    issuer=None
    for item in f: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        issuer=get_issuer(txt)
        if issuer is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            issuer=get_issuer(txt)
            
        if issuer is not None:
            break
    
    print isin, issuer
    if issuer is not None:
        issuerDic[isin]=issuer
    else:
        issuerDic[isin]=''

FR0013149382 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1390266838 SMBC NIKKO SECURITIES INC.
XS1378402199 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
CH0285315344 EXANE FINANCE
XS1138845844 MORGAN STANLEY B.V.
JP1023601G11 SG ISSUER
CH0254071894 LEONTEQ SECURITIES AG
XS1387752329 SMBC NIKKO SECURITIES INC.
XS1313827740 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1373121745 SMBC NIKKO SECURITIES INC.
XS1328130197 CHINA CONSTRUCTION BANK CORPORATION
XS1328604837 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1369058059 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1293806839 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS0532879300 BRITISH AND MALAYAN TRUSTEES LIMITED
XS1315761459 SMBC NIKKO SECURITIES INC.
XS1281070455 HSBC BANK PLC
XS1389980613 SMBC NIKKO SECURITIES INC.
XS1319973456 CREDIT SUISSE INTERNATIONAL
XS1314926939 SG ISSUER
XS1317103569 UBS AG, GOLDEN LANE
XS1338495218 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1338526970 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS0

In [11]:
de.to_csv('issuer.csv',issuerDic)

In [ ]:
# this is 88% !!!!